In [20]:
from TrotterExample import get_noise_model, executor, get_backend
from pyquil import Program
from pyquil.gates import H, CNOT, Z, MEASURE, S, X, Y, I, RX, RZ, FENCE
from pyquil.quilbase import Declare
from pyquil.api import local_forest_runtime
from pyquil.quantum_processor import QCSQuantumProcessor
with local_forest_runtime():

    # Here the backend for the simulation is prepared
    from pyquil.quantum_processor import NxQuantumProcessor
    from pyquil.noise import NoiseModel
    from pyquil import get_qc
    import networkx as nx
    backend = get_qc("5q-qvm") #str(n)+'q-qvm' #args.backend
    isa = backend.to_compiler_isa()
    backend_qubits = sorted(int(k) for k in isa.qubits.keys())
    # By default every qubit on our fake backend is connected with every other qubit. This breaks the algorhythm. So we need to change the topology
    # So this line here changes the topology of the fake backend. The condition in the end determines it
    edges = [(q1, q2) for q1 in backend_qubits for q2 in backend_qubits if abs(q1-q2) == 1]
    # Build the NX graph
    topo = nx.from_edgelist(edges)
    # You would uncomment the next line if you have disconnected qubits
    # topo.add_nodes_from(qubits)
    nx_quantum_processor = NxQuantumProcessor(topo)
    isa = nx_quantum_processor.to_compiler_isa()
    #quantum_processor.noise_model = decoherence_noise_with_asymmetric_ro(quantum_processor.to_compiler_isa())
    noise_model = NoiseModel([CNOT], [0.1,0.9])
    quantum_processor = QCSQuantumProcessor("test", isa, noise_model)
    #quantum_processor.noise_model = noise_model
    backend.compiler.quantum_processor = quantum_processor
    #print(quantum_processor.qubit_topology())
    # Here the backend without noise is prepared for later comparison
    #import qiskit.providers.fake_provider as fake_provider # FakeMelbourneV2, FakeCasablancaV2, FakeVigoV2, FakeLagosV2, FakeGuadalupeV2, FakeGuadalupe, FakeGeneva

    prog = Program()
    prog += Declare("ro", "BIT", 5)
    prog += CNOT(0,1)
    prog += MEASURE(0, ("ro", 0))
    prog += MEASURE(1, ("ro", 1))

    #backend = Aer.get_backend('qasm_simulator')
    #(noise_model, twoqubit_error_template, singlequbit_error_template) = get_noise_model()
    noise_model = NoiseModel([CNOT], [0.1,0.9])
    counts = executor([prog], backend, 1024)
    print(counts)
    print(noise_model)

AttributeError: 'CompilerISA' object has no attribute 'architecture'